# SML 301

## Session 8: Nearest Neighbors

* clustering
* KNN (K Nearest Neighbors)
* imputation

In [ ]:
# install packages (if need be)
#%pip install palmerpenguins

In [ ]:
# load libraries
import matplotlib.cm as cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.pyplot import subplots
from palmerpenguins import load_penguins
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans
from sklearn.impute import KNNImputer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier as KNN
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
# load data
penguins = load_penguins()

df = penguins[['species', 'body_mass_g', 'flipper_length_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['body_mass_g', 'flipper_length_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], color = "black")

ax.set_title("Training Data (seeking structure)")
ax.set_xlabel("body mass (g)")
ax.set_ylabel("flipper length (mm)")

plt.show()

# Clustering

* $k$ cluster centers are randomly created
* for each observation, assign it a label by the closest center by *distance*
* redefine centers as group means
* repeat until there are no changes in membership

In [ ]:
# Machine Learning with Python Cookbook
cluster = KMeans(n_clusters = 3, n_init = "auto", random_state = 301)
model = cluster.fit(X_train)

# view cluster centers
model.cluster_centers_

In [ ]:
centers_df = pd.DataFrame(model.cluster_centers_)
centers_df["id"] = centers_df.index + 1
#colors_array = ['red', 'blue', 'green']

fig, ax = plt.subplots()
ax.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], color = "#aaaaaa")
ax.scatter(centers_df.iloc[:, 0], centers_df.iloc[:, 1], c = centers_df.id, cmap = 'Set1', s = 301)

ax.set_title("K Centers")
ax.set_xlabel("body mass (g)")
ax.set_ylabel("flipper length (mm)")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = y_train)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")

ax2.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = model.labels_, cmap = 'Set1')
ax2.set_title("clustering predictions")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")

plt.show()

In [ ]:
y_pred = model.predict(X_test)
acc_train = np.mean(model.labels_ == y_train)
acc_test  = np.mean(y_pred == y_test)
print(f'train accuracy: {acc_train:.4f}')
print(f'test accuracy: {acc_test:.4f}')

## Scaling Numerical Variables

In [ ]:
df = penguins[['species', 'body_mass_g', 'flipper_length_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['body_mass_g', 'flipper_length_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

# rescale variables
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns = X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

In [ ]:
# Machine Learning with Python Cookbook
cluster = KMeans(n_clusters = 3, n_init = "auto", random_state = 301)
model = cluster.fit(X_train)

# view cluster centers
model.cluster_centers_

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = y_train)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")

ax2.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = model.labels_, cmap = 'Set1')
ax2.set_title("clustering predictions")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")

plt.show()

In [ ]:
y_pred = model.predict(X_test)
acc_train = np.mean(model.labels_ == y_train)
acc_test  = np.mean(y_pred == y_test)
print(f'train accuracy: {acc_train:.4f}')
print(f'test accuracy: {acc_test:.4f}')

## Another Example

In [ ]:
df = penguins[['species', 'flipper_length_mm', 'bill_length_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['flipper_length_mm', 'bill_length_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

# rescale variables
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns = X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

cluster = KMeans(n_clusters = 3, n_init = "auto", random_state = 301)
model = cluster.fit(X_train)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['flipper_length_mm'], X_train['bill_length_mm'], c = y_train)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")

ax2.scatter(X_train['flipper_length_mm'], X_train['bill_length_mm'], c = model.labels_, cmap = 'Set1')
ax2.set_title("clustering predictions")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")

plt.show()

In [ ]:
# TODO: what if model labels don't match LabelEncoder values?  Hungarian Algorithm
y_pred = (model.predict(X_test) + 2) % 3 #hacked the alignment
acc_train = np.mean((model.labels_  + 2) % 3 == y_train)
acc_test  = np.mean(y_pred == y_test)
print(f'train accuracy: {acc_train:.4f}')
print(f'test accuracy: {acc_test:.4f}')

## Choosing the number of clusters

In [ ]:
# all numerical predictors
df = penguins[['species', 'flipper_length_mm', 'body_mass_g', 'bill_length_mm', 'bill_depth_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['flipper_length_mm', 'body_mass_g', 'bill_length_mm', 'bill_depth_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

# rescale variables
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns = X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

#cluster = KMeans(n_clusters = 3, n_init = "auto", random_state = 301)
#model = cluster.fit(X_train)
#model.get_params()


### Scree Plot ("Elbow")

In [ ]:
k_vals = [2, 3, 4, 5, 6, 7]
SSE_ratios = np.full(len(k_vals), np.nan)

for k in k_vals:
    cluster = KMeans(n_clusters = k, n_init = "auto", random_state = 301)
    model = cluster.fit(X_train)
    SSE_ratios[k-2] = model.inertia_

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_vals, SSE_ratios)

ax.legend()
ax.set_title("Scree Plot")
ax.set_xlabel("number of clusters")
ax.set_ylabel("total within sum of squares")
ax.set_xticks(range(2,7))

plt.show()

### Silhouette Analysis

"The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster." 

https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

In [ ]:
k_vals = [2, 3, 4, 5, 6, 7]

for n_clusters in k_vals:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=301)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
       X.iloc[:, 0], X.iloc[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d" %n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

## Modern Approaches

### Batch Processing

In [ ]:
cluster = MiniBatchKMeans(n_clusters = 3, n_init = "auto", random_state = 301,
  batch_size = 50) #work with data in smaller batches
model = cluster.fit(X_train)

### DBSCAN

"Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density."

In [ ]:
cluster = DBSCAN(eps = 3, #max distance (from center)
  min_samples = 5,
  metric = 'euclidean')
model = cluster.fit(X_train)

### Hierarchical Merging

In [ ]:
cluster = AgglomerativeClustering(n_clusters = 3)
model = cluster.fit(X_train)

# K Nearest Neighbors

## Neighbors (and distances)

In [ ]:
df = penguins[['species', 'body_mass_g', 'flipper_length_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['body_mass_g', 'flipper_length_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

In [ ]:
# Machine Learning with Python Cookbook
nearest_neighbors = NearestNeighbors(n_neighbors = 10).fit(X_train)

new_penguin = [4500, 201]
distances, indices = nearest_neighbors.kneighbors([new_penguin])
neighbors_df = X_train.iloc[indices[0], :]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = y_train)
ax1.scatter(new_penguin[0], new_penguin[1], color = "black", marker = "X", s = 301)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")
ax1.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax1.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

ax2.scatter(neighbors_df.iloc[:, 0], neighbors_df.iloc[:, 1])
ax2.set_title("nearest neighbors")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")
ax2.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax2.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

plt.show()

## Standardization

In [ ]:
df = penguins[['species', 'body_mass_g', 'flipper_length_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['body_mass_g', 'flipper_length_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

# rescale variables
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns = X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

In [ ]:
# Machine Learning with Python Cookbook
nearest_neighbors = NearestNeighbors(n_neighbors = 10, metric = 'euclidean').fit(X_train)

new_penguin = [0, 0]
distances, indices = nearest_neighbors.kneighbors([new_penguin])
neighbors_df = X_train.iloc[indices[0], :]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = y_train)
ax1.scatter(new_penguin[0], new_penguin[1], color = "black", marker = "X", s = 301)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")
ax1.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax1.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

# TODO: don't hard code radius (max distance value) or circle center
ax1.add_patch(patches.Circle((0,0), 0.32, alpha = 0.301, color = "orange", fill = True))

ax2.scatter(neighbors_df.iloc[:, 0], neighbors_df.iloc[:, 1])
ax2.set_title("nearest neighbors")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")
ax2.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax2.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

plt.show()

## L1 Norm

In [ ]:
# Manhattan norm
nearest_neighbors = NearestNeighbors(n_neighbors = 10, metric = 'manhattan').fit(X_train)

new_penguin = [0, 0]
distances, indices = nearest_neighbors.kneighbors([new_penguin])
neighbors_df = X_train.iloc[indices[0], :]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.scatter(X_train['body_mass_g'], X_train['flipper_length_mm'], c = y_train)
ax1.scatter(new_penguin[0], new_penguin[1], color = "black", marker = "X", s = 301)
ax1.set_title("training data")
ax1.set_xlabel("body mass (g)")
ax1.set_ylabel("flipper length (mm)")
ax1.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax1.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

# TODO: don't hard code radius (max distance value)
ax1.add_patch(patches.Rectangle((-0.32,-0.32), 0.64, 0.64, alpha = 0.301, color = "orange", fill = True))

ax2.scatter(neighbors_df.iloc[:, 0], neighbors_df.iloc[:, 1])
ax2.set_title("nearest neighbors")
ax2.set_xlabel("body mass (g)")
ax2.set_ylabel("flipper length (mm)")
ax2.set_xlim(min(X_train['body_mass_g']), max(X_train['body_mass_g']))
ax2.set_ylim(min(X_train['flipper_length_mm']), max(X_train['flipper_length_mm']))

plt.show()

## KNN

In [ ]:
model = KNN(n_neighbors = 3, n_jobs = -1).fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
acc_train = np.mean(y_pred_train == y_train)
acc_test  = np.mean(y_pred_test == y_test)
print(f'train accuracy: {acc_train:.4f}')
print(f'test accuracy: {acc_test:.4f}')

### Choosing the number of neighbors

In [ ]:
k_vals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
acc_train = np.full(len(k_vals), np.nan)
acc_test = np.full(len(k_vals), np.nan)

for k in k_vals:
    model = KNN(n_neighbors = k, n_jobs = -1).fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    acc_train[k-1] = np.mean(y_pred_train == y_train)
    acc_test[k-1] = np.mean(y_pred_test == y_test)

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_vals, acc_train, label = "training accuracy")
ax.plot(k_vals, acc_test, label = "testing accuracy")

ax.legend()
ax.set_title("KNN parameter search")
ax.set_xlabel("number of neighbors")
ax.set_ylabel("accuracy")
ax.set_xticks(range(1,10))

plt.show()

### Grid Search

In [ ]:
# define model
knn = KNN(n_neighbors = 5, n_jobs = -1)

# define pipeline
pipeline = Pipeline([("knn", knn)])

# define search space (Python dictionary)
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "knn__metric": ['euclidean', 'manhattan']}]

# perform cross-validation and grid search
classifier = GridSearchCV(pipeline, search_space, cv = 10, verbose = 0).fit(X_train, y_train)

In [ ]:
# retrieve best parameters
classifier.best_estimator_.get_params()

## Imputation

In [ ]:
df = penguins[['species', 'body_mass_g', 'flipper_length_mm', 'bill_length_mm', 'bill_depth_mm']] #load in needed columns
df = df.dropna() #then remove missing data
X = df[['body_mass_g', 'flipper_length_mm', 'bill_length_mm', 'bill_depth_mm']] #explanatory variables
y = df['species'] #target variable
y = LabelEncoder().fit_transform(y)

# rescale variables
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns = X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 301)

In [ ]:
new_penguin = np.array([[0.6, 0.7, np.nan, np.nan]]) #standardized values
new_penguin_df = pd.DataFrame(new_penguin, columns = X_train.columns)

X_train_2 = pd.concat([X_train, new_penguin_df])
print(X_train_2.tail())

In [ ]:
X_train_imputed = KNNImputer(n_neighbors = 4).fit_transform(X_train_2)
X_train_imputed = pd.DataFrame(X_train_imputed, columns = X_train.columns)
print(X_train_imputed.tail())